# 1. erstellung tabelle mit checkliste und freitext für vergleich der landesportale Bayern und BW 

In [1]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Features nach Kategorien
features = {
    "Allgemeine Struktur & Zielgruppen": [
        ("Zielgruppen", "FT"),
        ("Thematische Gliederung / Navigation", "FT"),
        ("Sprachliche Zugänglichkeit: Leichte Sprache", "CB"),
        ("Sprachliche Zugänglichkeit: Mehrsprachigkeit", "CB & FT"),
        ("Sprachliche Zugänglichkeit: Barrierefreiheit", "CB"),
    ],
    "Daten & Inhalte": [
        ("Texte", "CB"),
        ("Bilder", "CB"),
        ("Karten", "CB"),
        ("Geodaten", "CB"),
        ("Archivmaterial", "CB"),
        ("Oral History", "CB"),
        ("Filme", "CB"),
        ("Verfügbarkeit von Metadaten & Qualität", "CB"),
        ("Suchbarkeit", "CB"),
        ("Lizenzierung", "FT"),
        ("Aktualität / Versionierung", "FT"),
    ],
    "Technische Features": [
        ("Suchfunktion: einfache und erweiterte Suche; Filtermöglichkeiten", "CB & FT"),
        ("Karten / Geobasierte Visualisierung", "CB"),
        ("Schnittstellen / APIs", "CB"),
        ("Nutzerkonto", "CB"),
        ("Nutzerkonto: Favoriten", "CB"),
        ("Nutzerkonto: Merklisten", "CB"),
        ("Nutzerkonto: Benachrichtigungen", "CB"),
        ("Mobilnutzung", "CB"),
        ("Barrierefreiheit / Zugang", "CB"),
    ],
    "Interaktivität & Beteiligung": [
        ("Feedback-Möglichkeiten", "CB"),
        ("Kommentare", "CB"),
        ("Nutzerbeiträge", "CB"),
    ],
    "Organisation & Betrieb": [
        ("Trägerschaft / Kooperationspartner", "FT"),
        ("Finanzierung: Start- und laufende Kosten, Fördermittel, Budgetmodell", "FT"),
        ("Personelle Ressourcen", "FT"),
        ("Technische Infrastruktur", "FT"),
        ("Standards & Normen", "FT"),
    ],
    "Mehrwertfunktionen": [
        ("Visualisierungen (Zeitleisten, Karten, Diagramme etc.)", "CB"),
        ("Bildungsangebote", "CB"),
        ("Personalisierte Empfehlungen / Content Vorschläge", "CB"),
        ("Suche nach Orten / Objekten auf Karte", "CB"),
        ("Integration mit sozialen Medien", "CB"),
    ],
    "Nachhaltigkeit & Weiterentwicklung": [
        ("Update- und Pflegezyklen; Verantwortlichkeiten", "CB"),
        ("Erweiterbarkeit (modularer Aufbau)", "CB"),
        ("Monitoring & Evaluation", "CB"),
        ("Rechtliche Rahmenbedingungen", "FT"),
    ]
}

# DataFrame vorbereiten
rows = []
for cat, feats in features.items():
    for feat, ftype in feats:
        rows.append([cat, feat, ftype, "", ""])  # leere Felder für Bayern und BaWü

df = pd.DataFrame(rows, columns=["Kategorie", "Feature", "Typ", "Bayern (bavarikon)", "Baden-Württemberg (Landeskunde)"])

# Speichern
df.to_csv("landesportale_matrix.csv", index=False, encoding="utf-8")
df.to_excel("landesportale_matrix.xlsx", index=False)

print("Dateien 'landesportale_matrix.csv' und 'landesportale_matrix.xlsx' wurden erstellt.")


Dateien 'landesportale_matrix.csv' und 'landesportale_matrix.xlsx' wurden erstellt.


Partner herausfinden bavaricon mit request

In [2]:
import requests
from bs4 import BeautifulSoup
import time

base_url = "https://www.bavarikon.de/partners"
params = {
    "query": "",
    "sort": "titel:asc,id:asc",
    "lang": "de",
    "offset": 0
}

partners = []

while True:
    r = requests.get(base_url, params=params)
    if r.status_code != 200:
        print(f"Fehler bei Offset {params['offset']}, Status: {r.status_code}")
        break

    soup = BeautifulSoup(r.text, "html.parser")

    # Partnernamen extrahieren
    items = soup.select("div.search_item h4 a")
    if not items:
        break

    for a in items:
        name = a.get_text(strip=True)
        partners.append(name)

    print(f"Offset {params['offset']} -> {len(items)} Partner gefunden.")

    # Nächste Seite (jeweils +50)
    params["offset"] += 50
    time.sleep(1)  # kleine Pause

# Alphabetisch sortieren und Duplikate vermeiden
partners = sorted(set(partners))

# TXT speichern
save_path = r"C:\Users\daniil\Desktop\Projekt landeskunde\data\bavarikon_partners.txt"
with open(save_path, "w", encoding="utf-8") as f:
    for p in partners:
        f.write(p + "\n")

print(f"Fertig! {len(partners)} Partner gespeichert in {save_path}")


Offset 0 -> 50 Partner gefunden.
Offset 50 -> 50 Partner gefunden.
Offset 100 -> 50 Partner gefunden.
Offset 150 -> 50 Partner gefunden.
Offset 200 -> 3 Partner gefunden.
Fertig! 203 Partner gespeichert in C:\Users\daniil\Desktop\Projekt landeskunde\data\bavarikon_partners.txt


# 2. Normalisierungstabelle für Postgresql aus HOL

In [1]:
import pandas as pd

# Vollständige Mapping-Daten für Landeskunde-Datenbank
mapping_data = [
    # Tabelle: orte
    {"Tabelle": "orte", "Feld": "id", "Datentyp": "SERIAL", "Beschreibung / Bedeutung": "Primärschlüssel (eindeutige ID)", "Quelle / Abschnitt": "-", "Bemerkung": "Automatisch generiert"},
    {"Tabelle": "orte", "Feld": "name", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Name des Ortes", "Quelle / Abschnitt": "1", "Bemerkung": "Verbindlicher Hauptname"},
    {"Tabelle": "orte", "Feld": "alternativenamen", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Andere Schreibweisen oder historische Varianten", "Quelle / Abschnitt": "4", "Bemerkung": '[{"name":"Arnsfelde","quelle":"1375"}]'},
    {"Tabelle": "orte", "Feld": "ortstyp", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Art der Siedlung (Dorf, Gut, Ansiedlung etc.)", "Quelle / Abschnitt": "1", "Bemerkung": "Grundtyp"},
    {"Tabelle": "orte", "Feld": "gemeinde_1900", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Gemeindezugehörigkeit im Jahr 1900", "Quelle / Abschnitt": "1", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "eingemeindungen", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Eingemeindungen und kommunale Veränderungen", "Quelle / Abschnitt": "1", "Bemerkung": '{"jahr":1928,"art":"Zusammenlegung","ziel":"Ort X"}'},
    {"Tabelle": "orte", "Feld": "gemarkung_1860", "Datentyp": "NUMERIC", "Beschreibung / Bedeutung": "Gemarkungsgröße im Jahr 1860 (ha)", "Quelle / Abschnitt": "2", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "gemarkung_1900", "Datentyp": "NUMERIC", "Beschreibung / Bedeutung": "Gemarkungsgröße im Jahr 1900 (ha)", "Quelle / Abschnitt": "2", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "gemarkung_1931", "Datentyp": "NUMERIC", "Beschreibung / Bedeutung": "Gemarkungsgröße im Jahr 1931 (ha)", "Quelle / Abschnitt": "2", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "siedlungsform", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Siedlungsform laut Urmesstischblatt o. ä.", "Quelle / Abschnitt": "3", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "flurnamen", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Erwähnte Flurnamen oder Flurformen", "Quelle / Abschnitt": "3", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "erste_erwaehnung", "Datentyp": "INT", "Beschreibung / Bedeutung": "Jahr der ersten schriftlichen Erwähnung", "Quelle / Abschnitt": "4", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "quelle_erwaehnung", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Quelle der ersten Erwähnung", "Quelle / Abschnitt": "4", "Bemerkung": "-"},
    {"Tabelle": "orte", "Feld": "gerichtsbarkeit", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Gerichtszugehörigkeit über die Zeit", "Quelle / Abschnitt": "5", "Bemerkung": '[{"von":1849,"bis":1879,"gericht":"Königl. Kreisgericht"}]'},
    {"Tabelle": "orte", "Feld": "wirtschaft", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Wirtschafts- und Sozialstruktur (Hufenzahl, Betriebe etc.)", "Quelle / Abschnitt": "7", "Bemerkung": '[{"jahr":1860,"hufen":28}]'},
    {"Tabelle": "orte", "Feld": "kirche", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Kirchliche Zugehörigkeit und Patronat", "Quelle / Abschnitt": "8", "Bemerkung": '[{"parochie":"Blumberg"}]'},
    {"Tabelle": "orte", "Feld": "baudenkmale", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Baudenkmale und künstlerische Hinweise", "Quelle / Abschnitt": "9", "Bemerkung": '[{"name":"Feldsteinkirche","stil":"frühgotisch"}]'},
    {"Tabelle": "orte", "Feld": "beschreibung", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Freitext-Beschreibung oder Zusammenfassung", "Quelle / Abschnitt": "alle", "Bemerkung": "Allgemeine Textbeschreibung"},
    {"Tabelle": "orte", "Feld": "geometrie", "Datentyp": "GEOMETRY(Point,4326)", "Beschreibung / Bedeutung": "Geografische Lage (Koordinaten)", "Quelle / Abschnitt": "-", "Bemerkung": "PostGIS-Feld"},

    # Tabelle: bevoelkerung
    {"Tabelle": "bevoelkerung", "Feld": "id", "Datentyp": "SERIAL", "Beschreibung / Bedeutung": "Primärschlüssel", "Quelle / Abschnitt": "-", "Bemerkung": "Automatisch generiert"},
    {"Tabelle": "bevoelkerung", "Feld": "ort_id", "Datentyp": "INT", "Beschreibung / Bedeutung": "Fremdschlüssel zu Orte", "Quelle / Abschnitt": "10", "Bemerkung": "-"},
    {"Tabelle": "bevoelkerung", "Feld": "jahr", "Datentyp": "INT", "Beschreibung / Bedeutung": "Jahr der Bevölkerungsangabe", "Quelle / Abschnitt": "10", "Bemerkung": "-"},
    {"Tabelle": "bevoelkerung", "Feld": "einwohnerzahl", "Datentyp": "INT", "Beschreibung / Bedeutung": "Einwohnerzahl des Jahres", "Quelle / Abschnitt": "10", "Bemerkung": "-"},

    # Tabelle: herrschaft
    {"Tabelle": "herrschaft", "Feld": "id", "Datentyp": "SERIAL", "Beschreibung / Bedeutung": "Primärschlüssel", "Quelle / Abschnitt": "-", "Bemerkung": "Automatisch generiert"},
    {"Tabelle": "herrschaft", "Feld": "ort_id", "Datentyp": "INT", "Beschreibung / Bedeutung": "Fremdschlüssel zu Orte", "Quelle / Abschnitt": "6", "Bemerkung": "-"},
    {"Tabelle": "herrschaft", "Feld": "besitzer", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Besitzer oder Herrschaft (Adel, Kloster, Amt)", "Quelle / Abschnitt": "6", "Bemerkung": "-"},
    {"Tabelle": "herrschaft", "Feld": "jahr_von", "Datentyp": "INT", "Beschreibung / Bedeutung": "Beginn der Besitzperiode", "Quelle / Abschnitt": "6", "Bemerkung": "-"},
    {"Tabelle": "herrschaft", "Feld": "jahr_bis", "Datentyp": "INT", "Beschreibung / Bedeutung": "Ende der Besitzperiode", "Quelle / Abschnitt": "6", "Bemerkung": "-"},

    # Tabelle: bilder
    {"Tabelle": "bilder", "Feld": "id", "Datentyp": "SERIAL", "Beschreibung / Bedeutung": "Primärschlüssel", "Quelle / Abschnitt": "-", "Bemerkung": "Automatisch generiert"},
    {"Tabelle": "bilder", "Feld": "ort_id", "Datentyp": "INT", "Beschreibung / Bedeutung": "Fremdschlüssel zu Orte", "Quelle / Abschnitt": "9", "Bemerkung": "-"},
    {"Tabelle": "bilder", "Feld": "url", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "URL oder Pfad zur Bilddatei", "Quelle / Abschnitt": "9", "Bemerkung": "auch lokal speicherbar"},
    {"Tabelle": "bilder", "Feld": "titel", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Titel des Bildes oder Plans", "Quelle / Abschnitt": "9", "Bemerkung": "-"},
    {"Tabelle": "bilder", "Feld": "typ", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Typ des Mediums (Foto, Plan, Karte etc.)", "Quelle / Abschnitt": "9", "Bemerkung": "-"},
    {"Tabelle": "bilder", "Feld": "beschreibung", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Beschreibung oder Quelle des Bildes", "Quelle / Abschnitt": "9", "Bemerkung": "-"},

    # Tabelle: diagramme
    {"Tabelle": "diagramme", "Feld": "id", "Datentyp": "SERIAL", "Beschreibung / Bedeutung": "Primärschlüssel", "Quelle / Abschnitt": "-", "Bemerkung": "Automatisch generiert"},
    {"Tabelle": "diagramme", "Feld": "ort_id", "Datentyp": "INT", "Beschreibung / Bedeutung": "Fremdschlüssel zu Orte", "Quelle / Abschnitt": "10", "Bemerkung": "-"},
    {"Tabelle": "diagramme", "Feld": "typ", "Datentyp": "TEXT", "Beschreibung / Bedeutung": "Typ des Diagramms (Bevölkerung, Wirtschaft etc.)", "Quelle / Abschnitt": "10", "Bemerkung": "-"},
    {"Tabelle": "diagramme", "Feld": "daten", "Datentyp": "JSONB", "Beschreibung / Bedeutung": "Daten für Diagramm (z. B. Zeitreihen)", "Quelle / Abschnitt": "10", "Bemerkung": '[{"jahr":1800,"wert":174}]'},
]

# DataFrame erstellen
df = pd.DataFrame(mapping_data)

# CSV speichern
df.to_csv("mapping_landeskunde.csv", index=False, encoding="utf-8")

# Excel speichern (optional)
df.to_excel("mapping_landeskunde.xlsx", index=False, engine="openpyxl")

print("Mapping-Dateien erstellt: mapping_landeskunde.csv und mapping_landeskunde.xlsx")


Mapping-Dateien erstellt: mapping_landeskunde.csv und mapping_landeskunde.xlsx
